# SQL на Python

Создаем соединение с нашей базой данных (их может быть больше одного). Если файла в скобках нет, он создается сам. <br> Я взяла тестовую базу данных <a href="https://github.com/lerocha/chinook-database/blob/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite">отсюда</a>.

In [ ]:
import sqlite3

conn = sqlite3.connect('Chinook_Sqlite.sqlite')

Создаем курсор - это специальный объект который делает запросы и получает их результаты

In [ ]:
cursor = conn.cursor()

<br>
## Добываем информацию из таблицы:
<b>SELECT column_name1, column_name2... FROM table</b> - получаем двумерный массив с инфой из нужных столбиков, а если нужна вся таблица, вместо названий столбиков можно написать <b>*</b>
<br>
<b>ORDER BY</b> - порядок, по какому столбику идет выравнивание
<br>
<b>LIMIT 3</b> - лимит количества выданных данных, в данном случае 3<br>
<b>OFFSET</b> - c какого значения отсчитывать лимит (например, три входа с 20 строчки)

In [ ]:
cursor.execute("SELECT Name FROM Artist ORDER BY Name LIMIT 3 OFFSET 1")

# Получаем результат сделанного запроса
results = cursor.fetchall()
results2 =  cursor.fetchall()

print(results)   # [('A Cor Do Som',), ('Aaron Copland & London Symphony Orchestra',), ('Aaron Goldberg',)]
print(results2)  # []

После получения результата из курсора, второй раз без повторения самого запроса егo получить нельзя — вернется пустой результат!

На выбор информации из таблицы можно накладывать ограничения с помощью <b>WHERE</b>, которые могут комбинироваться. В ограничениях работают симфолы =, !=, <, >, =>, >=.
А также <b>BETWEEN … AND … </b> <i>(col_name BETWEEN 1.5 AND 10.5)</i><br> 
<b>NOT BETWEEN … AND … </b> <i>(col_name NOT BETWEEN 1 AND 10)</i><br>
<b>IN (…) </b> <i>col_name IN (2, 4, 6)
                  col_name IN ("A", "B", "C")</i> - строка является одним из данных объектов<br>
<b>NOT IN (…) </b><i>col_name NOT IN (1, 3, 5)</i><br>
<b>(NOT) LIKE</b> <i> col_name (NOT) LIKE "ABC"</i><br> - то же, что и != и =
<b>%</b> <i>col_name LIKE "%AT%"</i> - получаем "AT", "ATTIC", "CAT" or even "BATS"<br>
<b>___</b> <i>col_name LIKE "AN_"</i> - получаем "AND", но не "AN", восстанавливаем один символ в любом месте<br>

In [ ]:
SELECT column, another_column, …
FROM mytable
WHERE condition
    AND/OR another_condition
    AND/OR …;
    
cursor.execute("SELECT id FROM movies WHERE id==6")

У execute() есть четкая регламентация порядка функций. GROUP BY идет только после WHERE, поэтому для того, чтобы добавить специфические условия для GROUP, мы можем использовать функцию <b>HAVING</b>:

In [ ]:
cursor.execute('''SELECT role, SUM(years_employed)
                FROM employees
                GROUP BY role
                HAVING role = "Engineer"''')

А вообще порядок такой:

In [ ]:
SELECT DISTINCT column, AGG_FUNC(column_or_expression), …
FROM mytable
    JOIN another_table
      ON mytable.column = another_table.column
    WHERE constraint_expression
    GROUP BY column
    HAVING constraint_expression
    ORDER BY column ASC/DESC
    LIMIT count OFFSET COUNT;

<br>
## Выражения в таблицах
В SELECT можно использовать математические выражение и оператор <b>AS</b>. Последнее можно использовать для того, чтобы переименовывать столбики более удобным образом. Можно использовать такие штуки, как:
<br><b>COUNT(*), COUNT(column)</b></br>
<br><b>MIN(column)</b></br>
<br><b>MAX(column)</b></br>
<br><b>AVG(column)</b></br>
<br><b>SUM(column)</b></br>

In [ ]:
# Table: movies (Read-only)
# id title        director       year   length_minutes
# 1  Toy Story    John Lasseter  1995   81
# 2  A Bug's Life John Lasseter  1998   95
# 3  Toy Story 2  John Lasseter  1999   93
# 4  Monsters     Pete Docter    2001   92
# 5  Finding Nemo Andrew Stanton 2003   107
...

# Table: boxoffice (Read-only)
# movie_id  rating    domestic_sales    international_sales
# 5         8.2       380843261         555900000
# 14        7.4       268492764         475066843
# 88        8         206445654         417277164
# 12        6.4       191452396         368400000
# 3         7.9       245852179         239163000
...

# Задача: вывести названия всех фильмов и ранжировать их по суммарным домашним и международным
# продажам в миллионах долларов.

cursor.select('''SELECT title, (domestic_sales + international_sales) / 1000000 AS gross_sales_millions
FROM movies
  JOIN boxoffice
    ON movies.id = boxoffice.movie_id''')

# title                gross_sales_millions
# Finding Nemo         936.743261
# Monsters University  743.559607
# Ratatouille          623.722818
# Cars 2               559.852396
# Toy Story 2          485.015179
# The Incredibles      631.442092

<br>
## Объединение данных таблиц

In [ ]:
#U) users               D) departments
#id name       d_id     id  name
#-- ----       ----     --  ----
# 1  Владимир    1       1  Сейлз
# 2  Антон       2       2  Поддержка
# 3  Александр   6       3  Финансы
# 4  Борис       2       4  Логистика
# 5  Юрий        4

Внутреннее объединение <b>INNER JOIN</b> (синоним JOIN, ключевое слово INNER можно опустить) - выбираются только совпадающие данные из объединяемых таблиц.
<br>
Чтобы получить данные, которые подходят по условию частично, необходимо использовать внешнее объединение - <b>OUTER JOIN</b>. Такое объединение вернет данные из обеих таблиц (совпадающие по условию объединения) ПЛЮС дополнит выборку оставшимися данными из внешней таблицы, которые по условию не подходят, заполнив недостающие данные значением NULL.

In [ ]:
cursor.execute('''SELECT u.id, u.name, d.name AS d_name 
               FROM users u 
               INNER JOIN departments d ON u.d_id = d.id''')

# В результате отсутствуют:
# - пользователь Александр (отдел 6 - не существует)
# - отдел Финансы (нет пользователей) 

# id name       d_name
# -- --------   ---------
# 1  Владимир    Сейлз
# 2  Антон       Поддержка
# 4  Борис       Поддержка
# 3  Юрий        Логистика

Существует два типа внешнего объединения OUTER JOIN - <b>LEFT OUTER JOIN</b> и <b>RIGHT OUTER JOIN</b>. Работают они одинаково, разница заключается в том что LEFT - указывает что "внешней" таблицей будет находящаяся слева (в нашем примере это таблица users).Ключевое слово OUTER можно опустить. Запись LEFT JOIN идентична LEFT OUTER JOIN.

In [ ]:
cursor.execute('''SELECT u.id, u.name, d.name AS d_name
                FROM users u
                LEFT OUTER JOIN departments d ON u.d_id = d.id''')

# Получаем полный список пользователей и сопоставленные департаменты.
# id      name          d_name
# --      --------      ---------
# 1       Владимир      Сейлз
# 2       Антон         Поддержка
# 3       Александр     NULL
# 4       Борис         Поддержка
# 5       Юрий          Логистика

Добавив условие <b>WHERE d.id IS NULL</b> в выборке останется только "3 Александр", так как у него не назначен департамент.<br> <b>RIGHT OUTER JOIN</b> вернет полный список департаментов (правая таблица) и сопоставленных пользователей.

In [ ]:
cursor.execute('''SELECT u.id, u.name, d.name AS d_name
            FROM users u
            RIGHT OUTER JOIN departments d ON u.d_id = d.id''')

# id      name        d_name
# --      --------    ---------
# 1       Владимир    Сейлз
# 2       Антон       Поддержка
# 4       Борис       Поддержка
# NULL    NULL        Финансы
# 5       Юрий        Логистика

<br>
## Запись в базу данных

In [ ]:
# Делаем INSERT запрос к базе данных, используя обычный SQL-синтаксис
cursor.execute("insert into Artist values (Null, 'A Aagrh!') ")

# Если мы не просто читаем, но и вносим изменения в базу данных - необходимо сохранить транзакцию
conn.commit()

# Проверяем результат
cursor.execute("SELECT Name FROM Artist ORDER BY Name LIMIT 3")
results = cursor.fetchall()
print(results)  # [('A Aagrh!',), ('A Cor Do Som',), ('Aaron Copland & London Symphony Orchestra',)]

Без <b>NULL</b> будет sqlite3.OperationalError: table Artist has 2 columns but 1 values were supplied<br><br>Если к базе установлено несколько соединений и одно из них осуществляет модификацю базы, то база SQLite залочивается до завершения (метод соединения <b>.commit()</b>) или отмены (метод соединения <b>.rollback()</b>) транзакции.
<br>
<br>
Также можно изменять информацию в уже существующих значениях с помощью <b>UPDATE</b>:

In [ ]:
UPDATE movies
SET title = "Toy Story 3", director = "Lee Unkrich"
WHERE id = 11;

И удалять с помощью <b>DELETE</b>:

In [ ]:
DELETE FROM movies
where year < 2005;

А еще добавлять новые стобики через <b>ALTER TABLE</b> с помощью <b>ADD</b>, удалять с помощью <b>DROP</b>, переименовывать с помощью <b>RENAME</b>:

In [ ]:
ALTER TABLE mytable
ADD column DataType OptionalTableConstraint 
    DEFAULT default_value;
    
ALTER TABLE mytable
DROP column_to_be_deleted;

ALTER TABLE mytable
RENAME TO new_table_name;

<br>
## Разбиваем запрос на несколько строк в тройных кавычках

Длинные запросы можно разбивать на несколько строк в произвольном порядке, если они заключены в тройные кавычки — одинарные ('''…''') или двойные ("""...""")

In [ ]:
cursor.execute("""
  SELECT name
  FROM Artist
  ORDER BY Name LIMIT 3
""")

Конечно, в таком простом примере разбивка не имеет смысла, но на сложных длинных запросах она может кардинально повышать читаемость кода.
<br>
<br>

## Объединяем запросы к базе данных в один вызов метода

Метод курсора <b>.execute()</b> позволяет делать только один запрос за раз, при попытке сделать несколько через точку с запятой будет ошибка. Вот так вот ничего не выйдет:

In [ ]:
cursor.execute("""
    insert into Artist values (Null, 'A Aagrh!');
    insert into Artist values (Null, 'A Aagrh-2!');
""")
# sqlite3.Warning: You can only execute one statement at a time.

Для решения такой задачи можно либо несколько раз вызывать метод курсора .execute(), либо использовать метод курсора <b>.executescript()</b>

In [ ]:
cursor.executescript("""
 insert into Artist values (Null, 'A Aagrh!');
 insert into Artist values (Null, 'A Aagrh-2!');
""")

Данный метод также удобен, когда у нас запросы сохранены в отдельной переменной или даже в файле и нам его надо применить такой запрос к базе.

## Подстановка значений в запрос
<b>Важно!</b> Никогда, ни при каких условиях, не используйте конкатенацию строк (+) или интерполяцию параметра в строке (%) для передачи переменных в SQL запрос. Такое формирование запроса, при возможности попадания в него пользовательских данных – это ворота для SQL-инъекций!
<br>
Правильный способ – использование второго аргумента метода .execute()
<br>
Возможны два варианта, но ни одним из этих способов не получится заменять имена таблиц: 

In [ ]:
# C подставновкой по порядку на места знаков вопросов:
cursor.execute("SELECT Name FROM Artist ORDER BY Name LIMIT ?", ('2'))

# И с использованием именнованных замен:
cursor.execute("SELECT Name from Artist ORDER BY Name LIMIT :limit", {"limit": 3})

<br>
## И многое другое, что можно делать с базой данных

Делаем множественную вставку строк проходя по коллекции с помощью метода курсора <b>.executemany()</b>

In [ ]:
# Обратите внимание, даже передавая одно значение - его нужно передавать кортежем!
# Именно по этому тут используется запятая в скобках!
new_artists = [
    ('A Aagrh!',),
    ('A Aagrh!-2',),
    ('A Aagrh!-3',),
]
cursor.executemany("insert into Artist values (Null, ?);", new_artists)

Получаем результаты по одному, используя метод курсора <b>.fetchone()</b>
<br>
Он всегда возвращает кортеж или <b>None</b>. если запрос пустой.<br>
<u><b>Важно!</b></u> Стандартный курсор забирает все данные с сервера сразу, не зависимо от того, используем мы .fetchall() или .fetchone()

In [ ]:
cursor.execute("SELECT Name FROM Artist ORDER BY Name LIMIT 3")
print(cursor.fetchone())    # ('A Cor Do Som',)
print(cursor.fetchone())    # ('Aaron Copland & London Symphony Orchestra',)
print(cursor.fetchone())    # ('Aaron Goldberg',)
print(cursor.fetchone())    # None

Курсор как итератор:

In [ ]:
# Использование курсора как итератора
for row in cursor.execute('SELECT Name from Artist ORDER BY Name LIMIT 3'):
        print(row)
# ('A Cor Do Som',)
# ('Aaron Copland & London Symphony Orchestra',)
# ('Aaron Goldberg',)

<br>
## Повышаем устойчивость кода 

Для большей устойчивости программы (особенно при операциях записи) можно оборачивать инструкции обращения к БД в блоки «try-except-else» и использовать встроенный в sqlite3 «родной» объект ошибок, например, так:

In [ ]:
try:
    cursor.execute(sql_statement)
    result = cursor.fetchall()
except sqlite3.DatabaseError as err:       
    print("Error: ", err)
else:
    conn.commit()

Создаем таблицы при помощи <b>CREATE TABLE</b>. В каждом столбике должен быть указан тип данных и могут быть свои ограничения, почитать об этом можно <a href="https://sqlbolt.com/lesson/creating_tables">тут.</a> Удаление происходить примерно так же <b>DROP TABLE</b>.

In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS mytable (
    column DataType TableConstraint DEFAULT default_value,
    another_column DataType TableConstraint DEFAULT default_value,
    …
    ''')

cursor.execute('''DROP TABLE IF EXISTS mytable''')